In [7]:
#import pandas for df management
import pandas as pd

# import numpy
import numpy as np

#import matplotlib for plotting funtionality
import matplotlib.pyplot as plt

# import citipy to get closest city from lat long
from citipy import citipy

# import the datetime module from the datetime library
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Import the API key.
from config import google_api_key

# import gmaps for google mapping
import gmaps

# Import the requests library.
import requests

# import os for path join
import os


In [9]:
cities_file_path = os.path.join("Data","WeatherPy_Database.csv")
city_data_df = pd.read_csv(cities_file_path)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date,Rainfall(1h),Snowfall(1h)
0,0,Fortuna,40.60,-124.16,70.00,72,1,14.99,clear sky,US,2020-07-09 21:13:14,0.00,0.0
1,1,Grindavik,63.84,-22.43,51.80,71,75,11.41,broken clouds,IS,2020-07-09 21:11:38,0.00,0.0
2,2,Komsomolskiy,40.43,71.72,77.00,33,100,6.93,overcast clouds,UZ,2020-07-09 21:13:15,0.00,0.0
3,3,Mataura,-46.19,168.86,42.06,92,100,5.06,light rain,NZ,2020-07-09 21:13:06,0.13,0.0
4,4,San Ignacio,17.16,-89.07,93.20,56,40,6.93,scattered clouds,BZ,2020-07-09 21:13:15,0.00,0.0


Get input from the user for the weather they would like for their trip.

In [16]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = str(input("Do you want it to be raining? (yes/no) "))
snowing = str(input("Do you want it to be snowing? (yes/no) "))
print(min_temp,max_temp,raining,snowing)


70.0 90.0 no no


In [23]:
if raining == "yes":
    if snowing == "yes":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                                (city_data_df["Max Temp"] >= min_temp) & \
                                                (city_data_df["Rainfall(1h)"] > 0) & \
                                                (city_data_df["Snowfall(1h)"] > 0)]
    elif snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                                (city_data_df["Max Temp"] >= min_temp) & \
                                                (city_data_df["Rainfall(1h)"] > 0) & \
                                                (city_data_df["Snowfall(1h)"] == 0)]
    else:
        print("Incorrect input for snow!")
elif raining == "no":
    if snowing == "yes":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                                (city_data_df["Max Temp"] >= min_temp) & \
                                                (city_data_df["Rainfall(1h)"] == 0) & \
                                                (city_data_df["Snowfall(1h)"] > 0)]
    elif snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                                (city_data_df["Max Temp"] >= min_temp) & \
                                                (city_data_df["Rainfall(1h)"] == 0) & \
                                                (city_data_df["Snowfall(1h)"] == 0)]
    else:
        print("Incorrect input for snow!")
else:
        print("Incorrect input for rain!")
preferred_cities_df.count()

City_ID         253
City            253
Lat             253
Lng             253
Max Temp        253
Humidity        253
Cloudiness      253
Wind Speed      253
Description     253
Country         253
Date            253
Rainfall(1h)    253
Snowfall(1h)    253
dtype: int64

In [18]:
hotel_df = preferred_cities_df[["City","Country","Description","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Description,Max Temp,Lat,Lng,Hotel Name
0,Fortuna,US,clear sky,70.00,40.60,-124.16,
2,Komsomolskiy,UZ,overcast clouds,77.00,40.43,71.72,
6,Matagami,CA,clear sky,86.00,49.75,-77.63,
11,Butaritari,KI,clear sky,83.89,3.07,172.79,
14,Saint-Philippe,RE,few clouds,71.60,-21.36,55.77,
...,...,...,...,...,...,...,...
617,Kanniyakumari,IN,overcast clouds,80.28,8.08,77.57,
618,Lorengau,PG,overcast clouds,82.60,-2.02,147.27,
620,Almenara,BR,scattered clouds,73.89,-16.18,-40.69,
621,Phan Thiet,VN,overcast clouds,76.71,10.93,108.10,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [20]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels ["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# Create the output file (CSV).
output_data_file = os.path.join("Data","WeatherPy_Vacation.csv")

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# set api key for gmaps
gmaps.configure(api_key=google_api_key)

# html for what goes in the descriptor box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a map of vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))